## An interactive Notebook simulating the time evolution of the Zeemann sublevel population in bosonic Dysprosium 164 under RF excitation

### README
Author: Maurice Rieger<br>
Date: 2023-05-23<br>

### About
Time evolution of the state population in bosonic Dysprosium 164 during RF-coupling. 

In [1]:
import sys
sys.path.append('../../')
import matplotlib 
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA 
import scipy.linalg as LA
from scipy.optimize import root_scalar
from scipy.optimize import curve_fit
from scipy import integrate
from scipy.stats import chi2
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from matplotlib.animation import FuncAnimation


#### First we define general constants: 

In [2]:
hbar = 6.62607015*10**(-34) / (2*np.pi)
h = 6.62607015*10**(-34)
e = 1.602176634*10**(-19) # Coulomb 
m_e = 9.1093837015*10**(-31) # kg 
mu_B = 5.7883818060*10**(-5) # in eV/T
mu_B_J = 9.2740100783*10**(-24) # in J/T
mu_0 = 1.25663706 * 10**(-6) #m kg s^-2 A^-2
k_B = 1.38064852* 10**(-23) # m^2 kg s^-2 K^-1
u = 1.66053906660*10**(-27) # kg 

#### We define constants of the Dy164 ground state

In [3]:
# Quantum numbers (orbital, spin, total): 
L = 6 
S = 2 
J = 8 

# Mass and g-factor: 
m = 162*u
g_J = 1+(J*(J+1)+S*(S+1)-L*(L+1))/(2*J*(J+1))

#### Here we define the system parameters: 

In [4]:
# Radio frequency field strength in Gauss: 
B_RF = 2.4/1000   

# Detuning from the resonance frequency in kHz: 
z = 0               


#### Now we define the Hamiltonian and the initial condition: 

In [5]:
# Calculate the Rabi frequency:  

Omega = g_J* mu_B_J/hbar *B_RF / 10**4  
Rabi = Omega/16 *np.sqrt(2*J) 
tvals = np.linspace(0,0.0005,1000)

# We define the Hamiltonian: 

def H_diag_real(J,z):
    Mat =  np.zeros(( int(2*J +1) ,int(2*J +1) ))
    for i in range( 0, int(2*J +1) ):
        mJ = J - i
        Mat[i,i] = - hbar * mJ * z * 1000
    return Mat

def J_l_offdiag_real(J,Omega_1):
    Mat =  np.zeros(( int(2*J+1) ,int(2*J+1) ))
    for i in range( 0, int(2*J) ):
        mJ = J - i
        Mat[i+1,i] = -Omega_1/4 *hbar * np.sqrt( J*(J+1) - mJ*(mJ-1) )
    return Mat

def J_r_offdiag_real(J,Omega_1):
    Mat = np.transpose(J_l_offdiag_real(J,Omega_1))
    return Mat

def H_real(J,z,Omega_1):
    Mat = J_l_offdiag_real(J,Omega_1) + J_r_offdiag_real(J,Omega_1) + H_diag_real(J,z)
    return Mat


# The system is initially in the ground state: 

initial = []

for i in range(0,17): 
    if i < 16: 
        initial.append(0)
    else:
        initial.append(1)

initial = np.array(initial)


## Interactive time evolution: 

In [6]:
def Plot_real(z,B_RF): 
    Omega = g_J* mu_B_J/hbar *B_RF / 10**4 /10**3 
    Rabi = Omega/16 *np.sqrt(2*J) 
    evals_real, evecs_real = LA.eigh(H_real(J,z,Omega))
    coefficients_real = evecs_real.conj().T @ initial 
    Probabilities_real = np.transpose([np.abs(evecs_real @ (np.exp(-1j*evals_real*t/hbar)*coefficients_real))**2 for t in tvals])

    fig = plt.figure()
    fig.set_size_inches(11,8)
    plt.rc('font', size=14)

    plt.plot(tvals*Rabi/np.pi,Probabilities_real[16], color = "indianred",label="$P=|<\psi|-8>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[15], color = "maroon",label="$P=|<\psi|-7>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[14], color = "orangered",label="$P=|<\psi|-6>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[13], color = "darkorange",label="$P=|<\psi|-5>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[12], color = "burlywood",label="$P=|<\psi|-4>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[11], color = "goldenrod",label="$P=|<\psi|-3>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[10], color = "palegreen",label="$P=|<\psi|-2>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[9], color = "darkseagreen",label="$P=|<\psi|-1>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[8], color = "forestgreen",label="$P=|<\psi|~~0~>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[7], color = "aquamarine",label="$P=|<\psi|+1>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[6], color = "lightblue",label="$P=|<\psi|+2>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[5], color = "turquoise",label="$P=|<\psi|+3>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[4], color = "steelblue",label="$P=|<\psi|+4>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[3], color = "midnightblue",label="$P=|<\psi|+5>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[2], color = "orchid",label="$P=|<\psi|+6>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[1], color = "indigo",label="$P=|<\psi|+7>|^2$")
    plt.plot(tvals*Rabi/np.pi,Probabilities_real[0], color = "black",label="$P=|<\psi|+8>|^2$")

    plt.legend(bbox_to_anchor=(1.03,1), loc="upper left")
    plt.ylabel(f"Probability of occupation")
    plt.xlabel('Time $[T_{Rabi}]$')
    plt.title(f"State population during a RF pulse over time (detuning: f={z} kHz)",fontsize=16)
    plt.grid(True)

    return()

interact(Plot_real, z=(-50,50,1), B_RF=(0,10,0.1) );

interactive(children=(IntSlider(value=0, description='z', max=50, min=-50), FloatSlider(value=5.0, description…